# Collection: copublications internationales (UE et hors UE)
* demande interne INRIA (27/07/2025)
* Réalisation du script (adaptation d'un ancien script) : Kumar Guha (Data DCIS/Inria)
* Date 27/02/2025, modifié le 29/08/2025. Denière version : 15/09/2025.

## Choix
* On ne retient que la première affiliation de chaque auteur (pas les niveaux supérieurs : exemple : Boston University School of Medicine et pas Boston University).
* Si un auteur rattaché à une structure française est aussi rattaché à une structure étrangère, on ne retient pas cet auteur pour compter une copublication internationale.
* Si un auteur de la structure Inria recherchée est aussi affilié à une autre strucutre étrangère, celle-ci n'est pas mentionnée.


## Étapes
* Extraire les publications des équipes concernées.
* identifier les publications dont les auteurs sont affiliés à un organisme étranger (hors France et DOM TOM)
* On crée des listes d'identifiants uniques pour les affiliations FR, Union Européenne et hors UE.
    *  on exclut les organismes étrangers dont les auteurs sont aussi affiliés à une structure FR
    *  on exclut les affiliations en double pour une même publication
    *  nettoyage des données.
* Génération d'un fichier Excel avec : chiffres, liste des publications, liste des organismes étrangers copubliants
* Première identification de la ville d'après 
    * dictionnaire déjà constitué par les recherches précédentes
    * nom de la ville entre crochets dans le nom de l'organisme


## Extraction des publications de HAL


In [ ]:
# 
###############################################################
## Extraction des publications de HAL
##############################################
import requests
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
from lxml import etree
import gc 
import locale
import re

###############################################################################################
##################### variables à modifier avant lancement du script ##########################
###############################################################################################

###########################################################
# Définir la structure recherchée:
###########################################################
nom_collection = "INRIA" # il s'agit des publications des équipes Inria de tous les centres.

# Identifiant de la structure "de réference" dont on analyse les publications (exemple Centre Inria de Rennes : 419153)
#(à chercher dans Aurehal : https://aurehal.archives-ouvertes.fr/structure/index)
i# Dictionnaire des structures
structures = {
    "419153": "Rennes",
    "104751": "Bordeaux",
    "34586": "Sophia",
    "2497": "Grenoble",
    "1096051": "Lyon",
    "129671": "Nancy",
    "104752": "Lille",
    "118511": "Saclay",
    "454310": "Paris",
    "1175218": "Paris(sorb)",
    "1225635": "Saclay (ipp)",
    "1225627": "Saclay (UPS)"
}

# Structures à exclure
# Galinette, Stack (id Aurehal :1088569,495900, 1088566, 525233 )
equipes_a_exclure = []

##########################################
# Définir la période recherchée
###########################################
annee_debut = 2018
annee_fin = 2025 # indiquer la même année si la recherche porte sur une seule année
pas = 3

def extraire_publications(id_aurehal_de_la_structure, nom_de_la_structure):
    for start_year in range(annee_debut, annee_fin + 1, pas):
        end_year = min(start_year + pas - 1, annee_fin)
        periode = f"[{start_year} TO {end_year}]"
        print(f"▶️ {nom_de_la_structure} ({id_aurehal_de_la_structure}) : Traitement de la période : {periode}")
        
        # Initialisation de toutes tes variables, comme dans ton script source
        params = {
            "q": f"publicationDateY_i:{periode}",
            "fq": f"structId_i:{id_aurehal_de_la_structure}",
            "wt": "xml-tei",
            "rows": 1,
            "sort": "docid asc"
        }
    #####################################################################
    ##################### script ########################################
    #####################################################################
    # Obtenir la date actuelle
    date_extraction_current = datetime.now().strftime("%Y-%m-%d")

    ## Spécifier le répertoire de log
    log_directory = '../log/'
    ## Créer le répertoire s'il n'existe pas
    os.makedirs(log_directory, exist_ok=True)


    # Configuration du logger
    log_file = date_extraction_current + '__international_publications_log.txt'
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

    # Configurer la localisation en français
    locale.setlocale(locale.LC_TIME, "French_France.1252")

    # La période est définie par les années saisies dans les variables au-dessus du script

    # periode = "[" + annee_debut + " TO " + annee_fin + "]"

    # variables cumulatives

    all_dataex = {}
    all_datafr = {}
    all_datapubli = []
    params = {}
    pas = 3

    for start_year in range(annee_debut, annee_fin + 1, pas):
        end_year = min(start_year + pas - 1, annee_fin)
        periode = f"[{start_year} TO {end_year}]"
        
        print(f"▶️ Traitement de la période : {periode}")
        
        params["q"] = f"publicationDateY_i:{periode}"
        
        # Réinitialise tes variables internes ici si besoin :
        cursor_mark = "*"
        previous_cursor_mark = None
        compteur = 0
        partenaire = 0
        dataex = []
        datafr = []
        datapubli = []
        unique_org_ex = {}
        unique_org_fr = {}


        # Fonction permettant de réessayer s'il n'y a pas de réponse
        def fetch_with_retry(url, params=None, max_retries=3, delay=2):
            """
            Effectue une requête GET avec plusieurs tentatives en cas d'échec.
            
            Args:
                url (str): L'URL de la requête.
                params (dict, optional): Paramètres de requête.
                max_retries (int): Nombre maximal de tentatives.
                delay (int): Temps d'attente entre chaque tentative (en secondes).

            Returns:
                Response object si la requête réussit, sinon None.
            """
            for attempt in range(max_retries):
                try:
                    response = requests.get(url, params=params, timeout=10)  # Timeout pour éviter les blocages
                    
                    if response.status_code == 200:
                        return response  # Succès
                    
                    print(f"⚠️ Tentative {attempt + 1} échouée ({response.status_code}). Nouvelle tentative...")

                except requests.RequestException as e:
                    print(f"⏳ Erreur réseau ({e}), tentative {attempt + 1}...")


        #### Processus de récupération de la liste des notices présentes dans HAL pour la période spécifiée
        # URL de base de l'API
        base_url = f"https://api.archives-ouvertes.fr/search/{nom_collection}?"

        # Paramètres de la requête : les résultats sont traités un par un en xml-tei
        params = {
            "q": f"publicationDateY_i:{periode}",
            "fq": f"structId_i:{id_aurehal_de_la_structure}",
            "wt": "xml-tei",
            "rows": 1,
            "sort": "docid asc"
        }

        # https://api.archives-ouvertes.fr/search/INRIA2?q=publicationDateY_i:[2019%20TO%202024]&fq=structId_i:(413916%20OR%20526070%20OR%20526181%20OR%20521735%20OR%20521714)&wt=xml-tei&rows=100&sort=docid%20asc

        # Initialisation du cursorMark (qui permet de réitérer la requête jusqu'à la fin des réponses de l'API)
        cursor_mark = "*"
        previous_cursor_mark = None

        # Définition des variables
        compteur = 0
        compte_publisUps = 0
        partenaire = 0
        dataex = []
        datafr = []
        datapubli = []
        unique_org_ex = {}
        unique_org_fr = {}
        # Exclure les DOM-TOM français
        France_et_dom_tom_codes = ['FR','GP', 'RE', 'MQ', 'GF', 'YT', 'PM', 'WF', 'TF', 'NC', 'PF']


        namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}


        #######################################
        # La requête est lancée en boucle et obtient un résultat (une notice) à chaque fois
        # chaque résultat est traité dans la boucle "while"
        ########################################
        while cursor_mark != previous_cursor_mark:
            # Mise à jour du cursorMark
            params["cursorMark"] = cursor_mark
            #print(f"CursorMark: {cursor_mark}")
            compteur += 1
            if compteur % 5000 == 0 or compteur == 1:
                print(compteur)
                if compteur % 5000 == 0:
                    # Convertir les données collectées pour les organismes étrangers
                    dataex = list(unique_org_ex.values())
                    df_ex = pd.DataFrame(dataex)
                    # Convertir les données collectées pour les organismes français
                    datafr = list(unique_org_fr.values())
                    df_fr = pd.DataFrame(datafr)

                    # Liste des publications/logiciels de HAL
                    df_publis = pd.DataFrame(datapubli)
                    # df_ex.to_excel(f"df_ex_{compteur}.xlsx", index=False)
                    # df_fr.to_excel(f"df_fr_{compteur}.xlsx", index=False)
                    # df_publis.to_excel(f"df_publis_{compteur}.xlsx", index=False)
                time.sleep(2)  # Pause de 2 secondes entre les requêtes
        
            # Limite pour tests
            # if compteur > 15:
            #     break

            response = fetch_with_retry(base_url, params)
            if response:
                try:
                    tree = etree.fromstring(response.content)
                except etree.XMLSyntaxError:
                    print("Erreur de syntaxe XML. Réponse non analysée.")
                    continue

                # Récupération de la valeur de next dans l'attribut de la première balise TEI
                next_cursor_mark = tree.attrib.get("next")
                # print(next_cursor_mark)
                
                # indication du nombre de notices répondant à la requête
                quantity_value = tree.find('.//tei:measure', namespaces=namespaces).attrib.get('quantity')
                
                if cursor_mark == "*":
                    # seulement lors de la première boucle, on indique le nombre total de notices répondant à la requête
                    print(f"nbre résultats : {quantity_value}. Durée estimée pour 4000 notices : 20 mn")

                # identification de la notice dans le xml-tei pour trouver les métadonnées
                biblfull_elements = tree.findall('.//tei:biblFull', namespaces=namespaces)

                if biblfull_elements:
                    biblfull = biblfull_elements[0]  # Get the first matching element
                else:
                    biblfull = None  # Handle the absence of the element
                    if cursor_mark == next_cursor_mark:
                        print(f"ancien curseur : {cursor_mark} - nouveau curseur : {next_cursor_mark}")
                        print("pas de biblFull - Terminé")
                        break
                    else:
                        print(f"Il y a eu un problème dans la réponse de HAL, veuillez relancer le script")
                        break

                # Récupération des metadonnées
                if biblfull is not None:
                    #identifiant de la publication dans HAL
                    halID = biblfull.xpath('.//tei:publicationStmt/tei:idno[@type="halId"]/text()', namespaces=namespaces) or ["pas de hal_ID"]
                    halID_value = halID[0] if halID else "no HalID"
                    # print(halID_value)
                    
                    # Domaines = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]/text()', namespaces=namespaces)
                    # Domaines_value = ";".join(domaine.strip() for domaine in Domaines if domaine)


                    # TRAITEMENT DES AFFILIATIONS contenues dans la notice
                    orgs = tree.findall('.//tei:listOrg[@type="structures"]/tei:org', namespaces=namespaces)

                    for org in orgs:
                        xml_id = org.xpath('@xml:id', namespaces=namespaces) # code de la structure
                        lenom = org.xpath('.//tei:orgName/text()', namespaces=namespaces)
                        lacronyme = org.xpath('.//tei:orgName[@type="acronym"]/text()', namespaces=namespaces)
                        lepays = org.xpath('.//tei:country/@key', namespaces=namespaces)
                        ladresse = [addr.text for addr in org.xpath('.//tei:addrLine', namespaces=namespaces) if addr.text]
                        ladresse_value = " ".join(ladresse)
                        lesrelations = org.xpath('.//tei:listRelation/tei:relation/@active', namespaces=namespaces) # codes des structures parentes

                        # Supprimer '#struct-' de chaque élément de la liste
                        lesrelations_cleaned = [relation.replace('#struct-', '') for relation in lesrelations]
                        xml_id_cleaned = xml_id[0].lstrip('struct-') 
                        
                        # si l'identifiant structure fait partie des identifiants à exclure on passe au suivant sans traiter.
                        # if xml_id_cleaned in equipes_a_exclure:
                        #     continue

                        # Organismes copubliants non français
                        if lepays and lepays[0] not in France_et_dom_tom_codes:

                            # print (f"{xml_id_cleaned} trouvé")
                            partenaire = 1 # à noter qu'il peut s'agir d'une structure mère étrangère qui ne va pas entrer en compte au final
                            unique_org_ex[xml_id[0]] = {
                                "Pays_ex": lepays,  # Le pays (on filtrera ensuite)
                                "OrganismeEx": lenom[0],  # Les noms des institutions
                                "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                                "adresse": ladresse_value,
                                "parents": lesrelations_cleaned

                            }
                        # Organisme FR
                        elif lepays and lepays[0] in France_et_dom_tom_codes:

                            #print(lepays)
                            unique_org_fr[xml_id[0]] = {
                                "Pays_fr": lepays,  # Le pays
                                "Organisme_fr": lenom[0],  # Les noms des institutions
                                "Acronyme_fr": lacronyme[0] if lacronyme else 'na',
                                "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                                "adresse": ladresse_value,
                                "parents": lesrelations_cleaned

                            }
                    
                    # Si on veut limiter les résultats aux publications avec des copubliants internationaux alors il faut décommenter la ligne suivante      
                    if partenaire == 1: # si on a trouvé un pays hors FR
                
                    # et il faut décaler les lignes suivantes aussi
                    # Sinon, on prend toutes les publications (par ex, si on veut calculer la proportion de copublications avec l'étranger par rapport au total)

                        # Récupération de l'année de publication   
                        
                        date_value = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct//tei:monogr/tei:imprint/tei:date[@type="datePub"]/text()', namespaces=namespaces)
                        date_produced = biblfull.xpath('.//tei:editionStmt/tei:edition/tei:date[@type="whenProduced"]/text()', namespaces=namespaces)
                        if date_value and date_value is not None:
                            date_text = date_value[0]  # Récupérer la chaîne de date
                            year_value = date_text[:4]  # Les 4 premiers caractères pour l'année
                        else:
                            year_value = date_produced[0][:4]

                        keywords = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:keywords/tei:term', namespaces=namespaces)
                        # Extraire les mots-clés et joindre avec ";"
                        keywords_str = ";".join(
                            " ".join(term.text.split())  # supprime espaces multiples et trims
                            for term in keywords
                            if term.text
                        )
                        # Récupérer tous les <classCode> avec scheme="halDomain"
                        hal_domain_elems = biblfull.xpath(
                            './/tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]',
                            namespaces=namespaces
                        )

                        # Nettoyer et joindre avec ";"
                        if hal_domain_elems:
                            hal_domain_str = ";".join(
                                elem.text.strip() for elem in hal_domain_elems if elem.text
                            )
                        else:
                            hal_domain_str = ""
                            
                        # Récupérer <abstract> en priorité "en", sinon "fr", sinon chaîne vide
                        def get_full_text(elem):
                            return "".join(elem.itertext()).strip() if elem is not None else ""
                        abstract_elem = biblfull.xpath(
                            './/tei:profileDesc/tei:abstract[@xml:lang="en"]',
                            namespaces=namespaces
                        )
                        if not abstract_elem:  # fallback en "fr"
                            abstract_elem = biblfull.xpath(
                                './/tei:profileDesc/tei:abstract[@xml:lang="fr"]',
                                namespaces=namespaces
                            )
                        abstract_str = get_full_text(abstract_elem[0]) if abstract_elem else ""
                        
                        # titre_revue = titre_journal[0].text if titre_journal  else ""
                        # # print(titre_revue)
                        # conference_titles = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct/tei:monogr/tei:meeting/tei:title', namespaces=namespaces)
                        # titre_conf = conference_titles[0].text if conference_titles else ""
                        # print(titre_conf)
                
                    
                    # Identification des affiliations associées à chaque auteur
                        for author in biblfull.xpath('.//tei:titleStmt/tei:author', namespaces=namespaces):
                            forename = author.xpath('.//tei:persName/tei:forename/text()', namespaces=namespaces)  or ["Unknown"]
                            surname = author.xpath('.//tei:persName/tei:surname/text()', namespaces=namespaces)  or ["Unknown"]
                                            
                            authorLastFirstnames = (f"{surname[0]}, {forename[0]}")

                            affiliations = author.xpath('.//tei:affiliation/@ref', namespaces=namespaces)

                            for affiliation in affiliations:
                                affiliation = affiliation.lstrip('#struct-')

                                # si l'identifiant structure fait partie des identifiants à exclure on passe au suivant sans traiter.
                                # if affiliation in equipes_a_exclure:
                                #     continue

                                datapubli.append ({
                                    "halID" : halID_value,
                                    "Auteur" : authorLastFirstnames,
                                    "affiliation" : affiliation,
                                    "Centre" : nom_struct if nom_struct else None
                                    "Annee" : year_value,
                                    "MotsCles": keywords_str,
                                    "Domaine(s)":hal_domain_str,
                                    "Resume":abstract_str,
                                })
                        partenaire = 0

                    #print(f"{halID_value} - {authorLastFirstnames} - {affiliation}")
                        
                    # En cas de récupération intensive de données, forcer la libération de la mémoire
                    # gc.collect()

                    # Mise à jour du cursorMark pour la prochaine itération
                    previous_cursor_mark = cursor_mark
                    cursor_mark = next_cursor_mark
                    # Pause pour éviter de surcharger l'API
                    # time.sleep(0.1)

                    if not next_cursor_mark:
                        print(compteur)
                        break

    # Ajoute les résultats cumulés
    all_dataex.update(unique_org_ex)
    all_datafr.update(unique_org_fr)
    all_datapubli.extend(datapubli)

for id_aurehal, nom_struct in structures.items():
    extraire_publications(id_aurehal, nom_struct)

print("Terminé")

In [ ]:
######## NOUVELLE VERSION ################
import requests
import time
import os
import pandas as pd
from datetime import datetime
import logging
from lxml import etree
import locale

# Dictionnaire des structures à interroger
structures = {
    "419153": "Rennes",
    "104751": "Bordeaux",
    "34586": "Sophia",
    "2497": "Grenoble",
    "1096051": "Lyon",
    "129671": "Nancy",
    "104752": "Lille",
    "118511": "Saclay",
    "454310": "Paris",
    "1175218": "Paris(sorb)",
    "1225635": "Saclay (ipp)",
    "1225627": "Saclay (UPS)"
}

nom_collection = "INRIA"
annee_debut = 2018
annee_fin = 2025
pas = 3

# Codes pays France et DOM-TOM pour filtrage
France_et_dom_tom_codes = ['FR','GP', 'RE', 'MQ', 'GF', 'YT', 'PM', 'WF', 'TF', 'NC', 'PF']

# Initialisation globale des cumuls des données
all_dataex = {}
all_datafr = {}
all_datapubli = []

# Configuration du logger (répertoire et fichier)
date_extraction_current = datetime.now().strftime("%Y-%m-%d")
log_directory = '../log/'
os.makedirs(log_directory, exist_ok=True)
log_file = os.path.join(log_directory, date_extraction_current + '__international_publications_log.txt')
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

# Configuration locale française
locale.setlocale(locale.LC_TIME, "French_France.1252")

def fetch_with_retry(url, params=None, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                return response
            print(f"⚠️ Tentative {attempt + 1} échouée ({response.status_code}). Nouvelle tentative...")
        except requests.RequestException as e:
            print(f"⏳ Erreur réseau ({e}), tentative {attempt + 1}...")
        time.sleep(delay)
    return None

def extraire_publications(id_aurehal, nom_struct):
    base_url = f"https://api.archives-ouvertes.fr/search/{nom_collection}?"
    for start_year in range(annee_debut, annee_fin + 1, pas):
        end_year = min(start_year + pas - 1, annee_fin)
        periode = f"[{start_year} TO {end_year}]"
        print(f"▶️ {nom_struct} ({id_aurehal}) : Traitement de la période : {periode}")

        params = {
            "q": f"publicationDateY_i:{periode}",
            "fq": f"structId_i:{id_aurehal}",
            "wt": "xml-tei",
            "rows": 1,
            "sort": "docid asc"
        }

        cursor_mark = "*"
        previous_cursor_mark = None
        compteur = 0
        partenaire = 0

        unique_org_ex = {}
        unique_org_fr = {}
        datapubli = []

        namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}

        while cursor_mark != previous_cursor_mark:
            params["cursorMark"] = cursor_mark
            compteur += 1
            if compteur % 5000 == 0 or compteur == 1:
                print(f"Nombre de notices traitées : {compteur}")
                # Optionnel: exporter ou sauvegarder les données partiellement ici

            response = fetch_with_retry(base_url, params)
            if not response:
                print("Échec de la récupération des données après plusieurs tentatives.")
                break

            try:
                tree = etree.fromstring(response.content)
            except etree.XMLSyntaxError:
                print("Erreur de syntaxe XML. Réponse non analysée.")
                continue

            next_cursor_mark = tree.attrib.get("next")
            quantity_value = tree.find('.//tei:measure', namespaces=namespaces).attrib.get('quantity') if tree.find('.//tei:measure', namespaces=namespaces) is not None else "0"

            if cursor_mark == "*":
                print(f"Nombre total de résultats pour cette période : {quantity_value}. Estimation durée: ~20 mn pour 4000 notices.")

            biblfull_elements = tree.findall('.//tei:biblFull', namespaces=namespaces)
            if not biblfull_elements:
                if cursor_mark == next_cursor_mark:
                    print("Aucune notice trouvée - fin de la récupération.")
                    break
                else:
                    print("Problème dans la réponse API, veuillez relancer.")
                    break
            biblfull = biblfull_elements[0]

            # Récupération id HAL
            halID = biblfull.xpath('.//tei:publicationStmt/tei:idno[@type="halId"]/text()', namespaces=namespaces) or ["pas de hal_ID"]
            halID_value = halID[0]

            # Traitement des affiliations
            orgs = tree.findall('.//tei:listOrg[@type="structures"]/tei:org', namespaces=namespaces)
            for org in orgs:
                xml_id = org.xpath('@xml:id', namespaces=namespaces)
                lenom = org.xpath('.//tei:orgName/text()', namespaces=namespaces)
                lacronyme = org.xpath('.//tei:orgName[@type="acronym"]/text()', namespaces=namespaces)
                lepays = org.xpath('.//tei:country/@key', namespaces=namespaces)
                ladresse = [addr.text for addr in org.xpath('.//tei:addrLine', namespaces=namespaces) if addr.text]
                ladresse_value = " ".join(ladresse)
                lesrelations = org.xpath('.//tei:listRelation/tei:relation/@active', namespaces=namespaces)

                lesrelations_cleaned = [relation.replace('#struct-', '') for relation in lesrelations]
                xml_id_cleaned = xml_id[0].lstrip('struct-') 

                # Exclusion non appliquée (décommenter si besoin)
                # if xml_id_cleaned in equipes_a_exclure:
                #     continue

                if lepays and lepays[0] not in France_et_dom_tom_codes:
                    partenaire = 1
                    unique_org_ex[xml_id[0]] = {
                        "Pays_ex": lepays,
                        "OrganismeEx": lenom[0] if lenom else '',
                        "ID_aurehal": xml_id_cleaned,
                        "adresse": ladresse_value,
                        "parents": lesrelations_cleaned
                    }
                elif lepays and lepays[0] in France_et_dom_tom_codes:
                    unique_org_fr[xml_id[0]] = {
                        "Pays_fr": lepays,
                        "Organisme_fr": lenom[0] if lenom else '',
                        "Acronyme_fr": lacronyme[0] if lacronyme else 'na',
                        "ID_aurehal": xml_id_cleaned,
                        "adresse": ladresse_value,
                        "parents": lesrelations_cleaned
                    }

            # Filtrer selon partenaire = 1 seulement, sinon prendre toutes les publications
            if partenaire == 1: 

                date_value = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct//tei:monogr/tei:imprint/tei:date[@type="datePub"]/text()', namespaces=namespaces)
                date_produced = biblfull.xpath('.//tei:editionStmt/tei:edition/tei:date[@type="whenProduced"]/text()', namespaces=namespaces)
                if date_value and date_value[0]:
                    year_value = date_value[0][:4]
                elif date_produced and date_produced[0]:
                    year_value = date_produced[0][:4]
                else:
                    year_value = ""

                keywords = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:keywords/tei:term', namespaces=namespaces)
                keywords_str = ";".join(
                    " ".join(term.text.split())
                    for term in keywords if term.text
                )

                hal_domain_elems = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]', namespaces=namespaces)
                hal_domain_str = ";".join(elem.text.strip() for elem in hal_domain_elems if elem.text) if hal_domain_elems else ""

                def get_full_text(elem):
                    return "".join(elem.itertext()).strip() if elem is not None else ""

                abstract_elem = biblfull.xpath('.//tei:profileDesc/tei:abstract[@xml:lang="en"]', namespaces=namespaces)
                if not abstract_elem:
                    abstract_elem = biblfull.xpath('.//tei:profileDesc/tei:abstract[@xml:lang="fr"]', namespaces=namespaces)
                abstract_str = get_full_text(abstract_elem[0]) if abstract_elem else ""

                # Extraction auteurs et affiliations
                for author in biblfull.xpath('.//tei:titleStmt/tei:author', namespaces=namespaces):
                    forename = author.xpath('.//tei:persName/tei:forename/text()', namespaces=namespaces) or ["Unknown"]
                    surname = author.xpath('.//tei:persName/tei:surname/text()', namespaces=namespaces) or ["Unknown"]
                    authorLastFirstnames = f"{surname[0]}, {forename[0]}"

                    affiliations = author.xpath('.//tei:affiliation/@ref', namespaces=namespaces)
                    for affiliation in affiliations:
                        affiliation = affiliation.lstrip('#struct-')
                        # if affiliation in equipes_a_exclure:
                        #     continue

                        datapubli.append({
                            "halID": halID_value,
                            "Auteur": authorLastFirstnames,
                            "affiliation": affiliation,
                            "Centre": nom_struct,
                            "Annee": year_value,
                            "MotsCles": keywords_str,
                            "Domaine(s)": hal_domain_str,
                            "Resume": abstract_str,
                        })
                partenaire = 0

            previous_cursor_mark = cursor_mark
            cursor_mark = next_cursor_mark

            if not next_cursor_mark:
                print(f"Fin des résultats pour {nom_struct} période {periode}, total notices traitées : {compteur}")
                break
            time.sleep(0.1)  # Pause courte pour ne pas surcharger

        all_dataex.update(unique_org_ex)
        all_datafr.update(unique_org_fr)
        all_datapubli.extend(datapubli)

# Boucle principale sur chaque structure au dictionnaire
for id_aurehal, nom_struct in structures.items():
    extraire_publications(id_aurehal, nom_struct)

print("Extraction terminée. Résultats cumulés dans all_dataex, all_datafr, all_datapubli")

#Temps de traitement : entre 4 et 5h.

▶️ Rennes (419153) : Traitement de la période : [2018 TO 2020]
Nombre de notices traitées : 1
Nombre total de résultats pour cette période : 2545. Estimation durée: ~20 mn pour 4000 notices.
Aucune notice trouvée - fin de la récupération.
▶️ Rennes (419153) : Traitement de la période : [2021 TO 2023]
Nombre de notices traitées : 1
Nombre total de résultats pour cette période : 2439. Estimation durée: ~20 mn pour 4000 notices.
Aucune notice trouvée - fin de la récupération.
▶️ Rennes (419153) : Traitement de la période : [2024 TO 2025]
Nombre de notices traitées : 1
Nombre total de résultats pour cette période : 1633. Estimation durée: ~20 mn pour 4000 notices.
Problème dans la réponse API, veuillez relancer.
▶️ Bordeaux (104751) : Traitement de la période : [2018 TO 2020]
Nombre de notices traitées : 1
Nombre total de résultats pour cette période : 1489. Estimation durée: ~20 mn pour 4000 notices.
Aucune notice trouvée - fin de la récupération.
▶️ Bordeaux (104751) : Traitement de la p

In [2]:
######################################################################
# Conversion en "dataframes" pour traitement des données et comptage
######################################################################

df_ex = pd.DataFrame(list(all_dataex.values()))
df_fr = pd.DataFrame(list(all_datafr.values()))
df_publis = pd.DataFrame(all_datapubli)

# Sauvegarde pour contrôle et tests
# df_ex.to_excel("df_ex_total.xlsx", index=False)
# df_fr.to_excel("df_fr_total.xlsx", index=False)
# df_publis.to_excel("df_publis_total.xlsx", index=False)
print("dataframes créés")


dataframes créés


In [3]:
# Sauvegarde pour contrôle et tests
df_ex.to_excel("df_ex_total.xlsx", index=False)
df_fr.to_excel("df_fr_total.xlsx", index=False)
df_publis.to_excel("df_publis_total.xlsx", index=False)
print("dataframes créés")

dataframes créés


In [ ]:
#  pour tests : df_publis=pd.read_excel("df_publis_total.xlsx")

In [4]:
df_publis.head(3)

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume
0,hal-01328959,"Soffer, Avy",452560,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...
1,hal-01328959,"Zhao, Xiaofei",525243,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...
2,hal-01229578,"Puy, Gilles",118587,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...


In [5]:

####################################
# FILTRE UE / hors UE pour df_ex
####################################
df_nonUE = ""
df_UE = ""
pays_UE = [
    "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR",
    "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL",
    "PL", "PT", "RO", "SK", "SI", "ES", "SE"
]

# On va créer 2 dataframes en fonction du pays de la structure (UE ou hors UE)
#

# Fonction pour vérifier si une valeur de la liste est dans la colonne `identifiant` ou `parents`
def filter_rows_EU(row):
    # Vérifie si une des valeurs de la liste se trouve dans `country` ou dans les valeurs de `parents`
    return any(country in pays_UE for country in row["Pays_ex"])

def filter_rows_ex(row):
    # Vérifie si une des valeurs de la liste ne se trouve pas dans `country` ou dans les valeurs de `parents`
    return any(country not in pays_UE for country in row["Pays_ex"])


# Filtrer les lignes du DataFrame pour garder celles des structures qui nous intéressent
df_UE = df_ex[df_ex.apply(filter_rows_EU, axis=1)]
df_UE.rename(columns={"OrganismeEx" : "Organisme_UE"}, inplace=True)


df_nonUE = df_ex[df_ex.apply(filter_rows_ex, axis=1)]
df_nonUE.rename(columns={"OrganismeEx" : "Organisme_Hors_UE"}, inplace=True)


print("Dataframes UE et non UE créés")


Dataframes UE et non UE créés


C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\458932396.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UE.rename(columns={"OrganismeEx" : "Organisme_UE"}, inplace=True)
C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\458932396.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonUE.rename(columns={"OrganismeEx" : "Organisme_Hors_UE"}, inplace=True)


In [6]:
###########################################################
# Interprétation des codes Pays en noms en toutes lettres 
###########################################################

# Récupérer les données de l'API
# url = "https://restcountries.com/v3.1/all"
# response = requests.get(url)
# countries_data = response.json()
import requests

def get_country_mapping():
    url = "https://restcountries.com/v3.1/all"
    params = {"fields": "cca2,name"}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            countries_data = response.json()
            if isinstance(countries_data, list):
                return {
                    country.get("cca2"): country.get("name", {}).get("common")
                    for country in countries_data
                    if country.get("cca2") and "name" in country and "common" in country["name"]
                }
            else:
                print("⚠️ Format inattendu :", type(countries_data))
                return {}
        else:
            print(f"⚠️ Erreur API ({response.status_code}): {response.json().get('message')}")
            return {}
    except Exception as e:
        print("❌ Erreur lors de la récupération des données pays :", e)
        return {}

# Utilisation
country_mapping = get_country_mapping()
print("✅ Exemple : FR →", country_mapping.get("FR"))  # Affiche 'France'




df_UE["Pays_ex"] = df_UE["Pays_ex"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_UE['TypePays'] = "EU"

df_nonUE["Pays_ex"] = df_nonUE["Pays_ex"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_nonUE['TypePays'] = "EX"

df_fr["Pays_fr"] = df_fr["Pays_fr"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_fr['TypePays'] = "FR"


# Afficher un aperçu du DataFrame modifié
print(df_UE.head(1))



✅ Exemple : FR → France
   Pays_ex                      Organisme_UE ID_aurehal  \
5  Germany  Institut für Mathematik [Berlin]       4560   

                                            adresse  parents TypePays  
5  Sekr. MA 4-1 Straße des 17.Juni 136 10623 Berlin  [86624]       EU  


C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\2021481470.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UE["Pays_ex"] = df_UE["Pays_ex"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\2021481470.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UE['TypePays'] = "EU"
C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\2021481470.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [7]:
########## 
# Ajouter l'acronyme des auteurs Inria dans la liste des publications
###########

# Charger ton fichier d'équipes Inria
df_equipes = pd.read_excel("equipesInriadeAurehal.xlsx")

# Vérifie que les colonnes utiles existent
assert "docid" in df_equipes.columns, "Colonne 'docid' manquante dans le fichier Excel"
assert "acronyme" in df_equipes.columns, "Colonne 'acronyme' manquante dans le fichier Excel"

# S'assurer que 'docid' et 'affiliation' sont bien de type str
df_equipes["docid"] = df_equipes["docid"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Fusion des deux DataFrames : on ajoute l'acronyme en fonction de l'affiliation
df_publis = df_publis.merge(df_equipes[["docid", "acronyme"]], how="left", left_on="affiliation", right_on="docid")

# Optionnel : supprimer la colonne docid (redondante après le merge)
df_publis.drop(columns=["docid"], inplace=True)

# Exemple d'affichage
print(df_publis[["halID", "Auteur", "affiliation", "acronyme"]].head(10))


          halID                 Auteur affiliation acronyme
0  hal-01328959            Soffer, Avy      452560      NaN
1  hal-01328959          Zhao, Xiaofei      525243   MINGUS
2  hal-01229578            Puy, Gilles      118587      NaN
3  hal-01229578            Puy, Gilles      210613   PANAMA
4  hal-01229578      Tremblay, Nicolas      210613   PANAMA
5  hal-01229578      Tremblay, Nicolas       35484      NaN
6  hal-01229578        Gribonval, Rémi      210613   PANAMA
7  hal-01229578  Vandergheynst, Pierre       35484      NaN
8  hal-01229578  Vandergheynst, Pierre      210613   PANAMA
9  hal-01389489       Bailleul, Ismaël          75      NaN


In [8]:
#  pour tests 
df_publis[df_publis["halID"] == "hal-00799242"]

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme
27305,hal-00799242,"Champagnat, Nicolas",206068,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,TOSCA
27306,hal-00799242,"Champagnat, Nicolas",211251,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,NaN
27307,hal-00799242,"Jabin, Pierre-Emmanuel",54156,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,NaN


In [9]:
# Nettoyage : Pour les auteurs Inria (auteurs ayant un acronyme), supprimer les autres affiliations
# Marquer les cas où pour chaque (halID, Auteur), au moins un acronyme est non-null
df=df_publis

df["has_acronyme"] = df.groupby(["halID", "Auteur"])["acronyme"].transform(lambda x: x.notna().any())

# Ne garder que :
# - les lignes où acronyme est non-null
# - ou les cas où aucune ligne pour ce (halID, Auteur) n'a d'acronyme
df = df[(df["acronyme"].notna()) | (~df["has_acronyme"])]

# Supprimer la colonne temporaire
df = df.drop(columns=["has_acronyme"])

df_publis = df
df_publis.head(10)


,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme
0,hal-01328959,"Soffer, Avy",452560,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,NaN
1,hal-01328959,"Zhao, Xiaofei",525243,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,MINGUS
3,hal-01229578,"Puy, Gilles",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA
4,hal-01229578,"Tremblay, Nicolas",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA
6,hal-01229578,"Gribonval, Rémi",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA
8,hal-01229578,"Vandergheynst, Pierre",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA
9,hal-01389489,"Bailleul, Ismaël",75,Rennes,2019,Paracontrolled calculus;Quaslilinear parabolic...,Mathematics [math]/Analysis of PDEs [math.AP],We provide in this work a local in time well-p...,NaN
10,hal-01389489,"Debussche, Arnaud",525243,Rennes,2019,Paracontrolled calculus;Quaslilinear parabolic...,Mathematics [math]/Analysis of PDEs [math.AP],We provide in this work a local in time well-p...,MINGUS
11,hal-01389489,"Hofmanova, Martina",4560,Rennes,2019,Paracontrolled calculus;Quaslilinear parabolic...,Mathematics [math]/Analysis of PDEs [math.AP],We provide in this work a local in time well-p...,NaN
12,hal-01390478,"Marschall, Tobias",92733,Rennes,2018,,Computer Science [cs]/Bioinformatics [q-bio.QM],"Many disciplines, from human genetics and onco...",NaN


In [10]:
# Ajouter l'organisme et le pays UE des auteurs à la liste générale des publications

# Vérifier que les colonnes existent
assert "ID_aurehal" in df_UE.columns, "Colonne 'ID_aurehal' manquante dans df_UE"
assert "Organisme_UE" in df_UE.columns, "Colonne 'Organisme_UE' manquante dans df_UE"
assert "Pays_ex" in df_UE.columns, "Colonne 'Pays_ex' manquante dans df_UE"
assert "adresse" in df_UE.columns, "Colonne 'adresse' manquante dans df_UE"

# Harmoniser les types de colonnes pour le merge
df_UE["ID_aurehal"] = df_UE["ID_aurehal"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Renommer les colonnes de df_nonUE pour éviter les collisions
df_UE_renamed = df_UE.rename(columns={
    "adresse": "adresse_UE",  # Pour éviter d’écraser la précédente
})

# Faire la jointure
df_publis = df_publis.merge(
    df_UE_renamed[["ID_aurehal", "Organisme_UE", "Pays_ex","adresse_UE"]],
    how="left",
    left_on="affiliation",
    right_on="ID_aurehal"
)

# Supprimer la colonne intermédiaire redondante
df_publis.drop(columns=["ID_aurehal"], inplace=True)

# Vérification du résultat
print(df_publis[["affiliation", "Organisme_UE", "Pays_ex","adresse_UE"]].head())


  affiliation Organisme_UE Pays_ex adresse_UE
0      452560          NaN     NaN        NaN
1      525243          NaN     NaN        NaN
2      210613          NaN     NaN        NaN
3      210613          NaN     NaN        NaN
4      210613          NaN     NaN        NaN


C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\2947255803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UE["ID_aurehal"] = df_UE["ID_aurehal"].astype(str)


In [ ]:
df_nonUE.head(1) #pour connaître le nom des colonnes afin de faire le traitement suivant


In [11]:
# Ajouter l'organisme et le pays Hors UE des auteurs

# Vérifier que les colonnes existent
assert "ID_aurehal" in df_nonUE.columns, "Colonne 'ID_aurehal' manquante dans df_nonUE"
assert "Organisme_Hors_UE" in df_nonUE.columns, "Colonne 'Organisme_Hors_UE' manquante dans df_nonUE"
assert "Pays_ex" in df_nonUE.columns, "Colonne 'Pays_ex' manquante dans df_nonUE"
assert "adresse" in df_nonUE.columns, "Colonne 'adresse' manquante dans df_nonUE"

# Harmoniser les types de colonnes pour le merge
df_nonUE["ID_aurehal"] = df_nonUE["ID_aurehal"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Renommer les colonnes de df_nonUE pour éviter les collisions
df_nonUE_renamed = df_nonUE.rename(columns={
    "Pays_ex": "Pays_ex_horsUE",  # Pour éviter d’écraser la précédente
    "adresse": "adresse_hors_UE", 
})

# Faire la jointure
df_publis = df_publis.merge(
    df_nonUE_renamed[["ID_aurehal", "Organisme_Hors_UE", "Pays_ex_horsUE","adresse_hors_UE"]],
    how="left",
    left_on="affiliation",
    right_on="ID_aurehal"
)

# Supprimer la colonne intermédiaire redondante
df_publis.drop(columns=["ID_aurehal"], inplace=True)

# Vérification du résultat
print(df_publis[["affiliation", "Organisme_Hors_UE", "Pays_ex_horsUE","adresse_hors_UE"]].head())

  affiliation          Organisme_Hors_UE Pays_ex_horsUE  \
0      452560  Department of Mathematics  United States   
1      525243                        NaN            NaN   
2      210613                        NaN            NaN   
3      210613                        NaN            NaN   
4      210613                        NaN            NaN   

                                     adresse_hors_UE  
0  Hill Center for the Mathematical Sciences110 F...  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                                NaN  


C:\Users\kguha\AppData\Local\Temp\ipykernel_5116\3645708706.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonUE["ID_aurehal"] = df_nonUE["ID_aurehal"].astype(str)


In [12]:
#  contrôle pour tests 
df_publis[df_publis["halID"] == "hal-00799242"]

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE
24088,hal-00799242,"Champagnat, Nicolas",206068,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,TOSCA,NaN,NaN,NaN,NaN,NaN,NaN
24089,hal-00799242,"Jabin, Pierre-Emmanuel",54156,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,NaN,NaN,NaN,NaN,Department of Mathematics [College Park],United States,"4176 Campus Drive - William E. Kirwan Hall, Co..."


In [13]:
##########################################
# Supprimer les auteurs HORS INRIA qui ont à la fois une affiliation française et une affiliation étrangère)
#########################################
# Indicateurs
df_publis["has_org"] = df_publis["Organisme_UE"].notna() | df_publis["Organisme_Hors_UE"].notna()
df_publis["has_no_acronyme"] = df_publis["acronyme"].isna() | (df_publis["acronyme"].str.strip() == "")

# Grouper par halID + Auteur
grouped = df_publis.groupby(["halID", "Auteur"]).agg(
    n_lignes=("halID", "count"),          # nombre de lignes pour ce couple
    has_no_acronyme=("has_no_acronyme", "any"),
    has_org=("has_org", "any")
).reset_index()

# On garde uniquement ceux qui ont au moins 2 lignes et les deux cas
auteurs_mixtes = grouped[
    (grouped["n_lignes"] >= 2) &
    (grouped["has_no_acronyme"]) &
    (grouped["has_org"])
]

# Supprimer ces auteurs de df_publis
df_publis_clean = df_publis.merge(
    auteurs_mixtes[["halID", "Auteur"]],
    on=["halID", "Auteur"],
    how="left",
    indicator=True
)
df_publis_clean = df_publis_clean[df_publis_clean["_merge"] == "left_only"].drop(columns="_merge")

print(f"✅ {len(df_publis) - len(df_publis_clean)} lignes supprimées")



✅ 32958 lignes supprimées


In [14]:
# contrôle pour test 
df_publis_clean[df_publis_clean["halID"] == "hal-00799242"]

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme
24088,hal-00799242,"Champagnat, Nicolas",206068,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,TOSCA,NaN,NaN,NaN,NaN,NaN,NaN,False,False
24089,hal-00799242,"Jabin, Pierre-Emmanuel",54156,Sophia,2018,pathwise uniqueness;rough drift;rough diffusio...,Mathematics [math]/Probability [math.PR],We study strong existence and pathwise uniquen...,NaN,NaN,NaN,NaN,Department of Mathematics [College Park],United States,"4176 Campus Drive - William E. Kirwan Hall, Co...",True,True


In [15]:
# contrôle pour test 
df_publis_clean[df_publis_clean["halID"] == "hal-01895279"]

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme
25377,hal-01895279,"Cabrera-Bosquet, Llorenç",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25378,hal-01895279,"Alvarez Prado, Santiago",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25379,hal-01895279,"Perez, Raphael",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25380,hal-01895279,"Artzet, Simon",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25381,hal-01895279,"Pradal, Christophe",141072,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,ZENITH,NaN,NaN,NaN,NaN,NaN,NaN,False,False
25382,hal-01895279,"Coupel-Ledru, Aude",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25383,hal-01895279,"Fournier, Christian",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
25384,hal-01895279,"Tardieu, Francois",37835,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True


In [16]:
df_publis = df_publis_clean

In [17]:
# Nettoyage : On supprime tous les co-auteurs français = ne sont pas Inria (pas d'acronyme) et n'ont pas d'affiliations étrangères

# Colonnes à tester pour le vide
cols_to_check = ["Organisme_UE", "Organisme_Hors_UE", "acronyme"]

# Masque des lignes vides
mask_empty = df_publis[cols_to_check].apply(
    lambda row: all(pd.isna(v) or str(v).strip() == "" for v in row),
    axis=1
)

# On garde seulement les lignes qui ne sont pas "vides"
df_publis_clean = df_publis[~mask_empty].copy()

print(f"✅ {mask_empty.sum()} lignes supprimées")



✅ 28754 lignes supprimées


In [18]:
# contrôle pour test 
df_publis_clean[df_publis_clean["halID"] == "hal-01895279"]

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme
25381,hal-01895279,"Pradal, Christophe",141072,Sophia,2019,Light use efficiency;Varietal mixture;Inter-ge...,Computer Science [cs]/Modeling and Simulation,Multi-genotype canopies are frequent in phenot...,ZENITH,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "cea-04228169"]

In [19]:
df_publis = df_publis_clean

In [20]:
df_publis.head(5)

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme
0,hal-01328959,"Soffer, Avy",452560,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,NaN,NaN,NaN,NaN,Department of Mathematics,United States,Hill Center for the Mathematical Sciences110 F...,True,True
1,hal-01328959,"Zhao, Xiaofei",525243,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,MINGUS,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,hal-01229578,"Puy, Gilles",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,hal-01229578,"Tremblay, Nicolas",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,hal-01229578,"Gribonval, Rémi",210613,Rennes,2018,,Engineering Sciences [physics]/Signal and Imag...,We study the problem of sampling k-bandlimited...,PANAMA,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [21]:
# On ne garde pas les publications avec juste un seul auteur
df_publis_clean = df_publis[df_publis.groupby("halID")["halID"].transform("count") > 1].copy()


In [22]:
# contrôle pour test 
df_publis_clean[df_publis_clean["halID"] == "hal-01895279"] # ne doit pas y être

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme


In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "hal-05212970"]

In [23]:
df_publis = df_publis_clean

In [69]:
df_publis.head(2)

,halID,Auteur,affiliation,Centre,Annee,MotsCles,Domaine(s),Resume,acronyme,Organisme_UE,Pays_ex,adresse_UE,Organisme_Hors_UE,Pays_ex_horsUE,adresse_hors_UE,has_org,has_no_acronyme
0,hal-01328959,"Soffer, Avy",452560,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,NaN,NaN,NaN,NaN,Department of Mathematics,United States,Hill Center for the Mathematical Sciences110 F...,True,True
1,hal-01328959,"Zhao, Xiaofei",525243,Rennes,2018,vortex;radiation;modulation equations;numerica...,Physics [physics]/Mathematical Physics [math-ph],We apply the modulation theory to study the vo...,MINGUS,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [70]:
# Fichier final (1e partie) avec , pour chaque auteur Inria, les copubliants étrangers

import pandas as pd

# 1. On part du df_publis et on isole les auteurs FR et étrangers
auteurs_fr = df_publis[pd.notna(df_publis['acronyme']) & (df_publis['acronyme'].str.strip() != '')]
auteurs_etr = df_publis[pd.isna(df_publis['acronyme']) | (df_publis['acronyme'].str.strip() == '')]

# 2. Pour chaque auteur étranger, on veut rattacher les auteurs FR du même halID
rows = []
for _, row_etr in auteurs_etr.iterrows():
    hal_id = row_etr['halID']
    # Trouver les auteurs FR liés à ce halID
    fr_list = auteurs_fr[auteurs_fr['halID'] == hal_id]
    for _, row_fr in fr_list.iterrows():
        rows.append({
            'Equipe': row_fr['acronyme'],
            'Centre':row_fr['Centre'],
            'Auteurs FR': row_fr['Auteur'],
            'Auteurs copubliants': row_etr['Auteur'],
            'Organisme copubliant': row_etr['Organisme_Hors_UE'] if pd.notna(row_etr['Organisme_Hors_UE']) else row_etr['Organisme_UE'],
            'Adresse': row_etr['adresse_hors_UE'] if pd.notna(row_etr['adresse_hors_UE']) else row_etr['adresse_UE'],
            'Pays': row_etr['Pays_ex_horsUE'] if pd.notna(row_etr['Pays_ex_horsUE']) else row_etr['Pays_ex'],
            'ID Aurehal': row_etr['affiliation'],
            'Année': row_etr['Annee'],
            'UE/Non UE': 'UE' if pd.notna(row_etr['Organisme_UE']) else 'Non UE',
            'HalID': hal_id,
            'Domaine(s)': row_etr['Domaine(s)'],
            'Mots-cles' : row_etr['MotsCles'],
            'Resume':row_etr['Resume'],
        })

# 3. Construire le DataFrame final
df_final = pd.DataFrame(rows)


# 4. Trier par Equipe, Auteurs FR, Auteurs copubliants
df_final = df_final.sort_values(by=['Equipe', 'Auteurs FR', 'Auteurs copubliants']).reset_index(drop=True)


# 5. Exporter vers Excel
nom_du_fichier = f"Copubliants_par_auteur_Inria_tout.xlsx"
df_final.to_excel(nom_du_fichier, index=False)

print(f"✅ Fichier Excel créé : {nom_du_fichier}")


✅ Fichier Excel créé : Copubliants_par_auteur_Inria_tout.xlsx


In [71]:
# contrôle pour test df_final[df_final["HalID"] == "hal-00799242"]
df_final.head(3)

,Equipe,Centre,Auteurs FR,Auteurs copubliants,Organisme copubliant,Adresse,Pays,ID Aurehal,Année,UE/Non UE,HalID,Domaine(s),Mots-cles,Resume
0,ABS,Sophia,"Cazals, Frédéric","Agashe, Viraj",Indian Institute of Technology Delhi,"Hauz Khas, New Delhi - 110 016. India",India,51173,2023,Non UE,hal-04366589,Computer Science [cs]/Bioinformatics [q-bio.QM],,Abstract Designing movesets providing high qua...
1,ABS,Sophia,"Cazals, Frédéric","Bahar, Ivet",University of Pittsburgh,"4200 Fifth Avenue Pittsburgh, PA 15260",United States,480689,2019,Non UE,hal-01968170,Computer Science [cs]/Computational Geometry [...,,
2,ABS,Sophia,"Cazals, Frédéric","Carazo, José Maria",Centro Nacional de Biotecnología [Madrid],"Campus de la Universidad Autónoma de Madrid, 2...",Spain,425523,2019,UE,hal-01968170,Computer Science [cs]/Computational Geometry [...,,


In [ ]:
# décommenter la ligne suivante ligne si la librairie geotext n'est pas installée 
# pip install geotext

In [41]:
print(nom_du_fichier)

Copubliants_par_auteur_Inria_tout.xlsx


In [ ]:
###################
# Premier repérage des villes :
#1 Utilisation du fichier contenant la liste déjà vérifiée des villes associées à un ID Aurehal = dictionnaire des villes aurehal

#####################""

import pandas as pd
from geotext import GeoText
import re

df_publis_tout = ""
###########################################
# Chargement du fichier des copublications
##########################################
df_publis_tout = pd.read_excel(nom_du_fichier)


#######################
# Renseigner avec le dictionnaire déjà existant
######################
df_villes =""
# Chargement du fichier ID Aurehal - Ville
df_villes = pd.read_excel("ID_Aurehal_Ville_Etat_Latitude_Longitude.xlsx")


# Fusionner les deux DataFrames sur la colonne ID_Aurehal
df_publis_tout = df_publis_tout.merge(
    df_villes[["ID_Aurehal","Ville","StateCode", "Latitude", "Longitude","geonameid"]],
    left_on="ID Aurehal", #nom de la colonne dans df_publis_tout
    right_on="ID_Aurehal",#nom de la colonne dans df_villes
    how="left"  # garde toutes les lignes de df_publis_tout, même si pas de correspondance
)

# Suppression de la colonne ID_Aurehal, qui ne nous sert plus
df_publis_tout = df_publis_tout.drop(columns=["ID_Aurehal"])

df_publis_tout.to_excel("resultat_avec_villes_du_dictionnaire.xlsx", index=False)

# temps de traitement normal environ 20 secondes

In [ ]:
# Pour contrôle visuel, il doit y avoir un résultat pour cette référence
df_publis_tout[df_publis_tout["HalID"] == "hal-05212970"]

,Centre,Equipe,Auteurs FR,Auteurs copubliants,Organisme copubliant,Adresse,Ville,Pays,ID Aurehal,UE/Non UE,Année,HalID,Domaine(s),Mots-cles,Resume,Latitude,Longitude,geonameid
12277,Grenoble,DATAMOVE,"Carastan-Santos, Danilo","Cordeiro, Daniel",Escola de Artes Ciências e Humanidades,"Av. Arlindo Béttio, 1000 Ermelino Matarazzo Sã...",None,Brazil,143207,Non UE,2025,hal-05212970,"Computer Science [cs]/Distributed, Parallel, a...",NaN,This work presents a carbon footprint plugin d...,NaN,NaN,NaN
12278,Grenoble,DATAMOVE,"Carastan-Santos, Danilo","Mazzini Bruschi, Sarita",Instituto de Ciências Mathemàticas e de Comput...,"Avenida Trabalhador São-carlense, 400 - Centro...",São Carlos,Brazil,265913,Non UE,2025,hal-05212970,"Computer Science [cs]/Distributed, Parallel, a...",NaN,This work presents a carbon footprint plugin d...,NaN,NaN,NaN
12281,Grenoble,DATAMOVE,"Carastan-Santos, Danilo","Saraiva, Gabriella",Escola de Artes Ciências e Humanidades,"Av. Arlindo Béttio, 1000 Ermelino Matarazzo Sã...",None,Brazil,143207,Non UE,2025,hal-05212970,"Computer Science [cs]/Distributed, Parallel, a...",NaN,This work presents a carbon footprint plugin d...,NaN,NaN,NaN


In [ ]:
#################################################################################
# 2e étape Identification des villes entre crochets dans le nom de l'organisme (il manquera encore la latitude et la longitude)
################################################################################

# Liste des pays à ignorer
pays_a_ignorer = {
    "Algeria", "Argentina", "Australia", "Austria", "Belgium", "Bolivia",
    "Bosnia and Herzegovina", "Brazil", "Brunei", "Bulgaria", "Burkina Faso",
    "Cameroon", "Canada", "Chile", "China", "Colombia", "Costa Rica", "Croatia",
    "Cyprus", "Czechia", "Denmark", "Ecuador", "Estonia", "Finland", "Georgia",
    "Germany", "Greece", "Hong Kong", "Hungary", "Iceland", "India", "Indonesia",
    "Iran", "Ireland", "Israel", "Italy", "Japan", "Jordan", "Kenya", "Latvia",
    "Lebanon", "Lithuania", "Luxembourg", "Madagascar", "Malaysia", "Malta",
    "Mexico", "Morocco", "Netherlands", "New Zealand", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Peru", "Poland", "Portugal",
    "Romania", "Russia", "Saudi Arabia", "Senegal", "Serbia", "Singapore",
    "Slovakia", "Slovenia", "South Africa", "South Korea", "Spain", "Sweden",
    "Switzerland", "Taiwan", "Thailand", "Tunisia", "Turkey", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay",
    "Venezuela", "Vietnam"
}

# Fonction pour extraire la ville entre crochets
def get_ville(organisme, adresse):
    if isinstance(organisme, str):
        match = re.search(r"\[(.*?)\]", organisme)
        if match:
            contenu = match.group(1).strip()
            if contenu not in pays_a_ignorer:
                return contenu
    return None

# Appliquer la fonction UNIQUEMENT si Ville est vide
mask = df_publis_tout["Ville"].isna() | (df_publis_tout["Ville"] == "")
df_publis_tout.loc[mask, "Ville"] = df_publis_tout[mask].apply(
    lambda row: get_ville(row["Organisme copubliant"], row["Adresse"]), axis=1
)

# Créer un DataFrame avec les lignes où une ville a été extraite
df_matches = df_publis_tout[df_publis_tout["Ville"].notna() &
                            (df_publis_tout["Ville"] != "")].copy()

# Garder uniquement les colonnes souhaitées et supprimer les doublons sur ID Aurehal
df_matches = df_matches[["ID Aurehal", "Organisme copubliant", "Ville"]].drop_duplicates(subset=["ID Aurehal"])

# Exporter la liste des ID avec Organisme copubliant et Ville
# on ajoutera les latitudes et longitudes déjà connues
# ensuite on vérifiera les inconnues pour voir si la ville est correctement identifiée
df_matches.to_excel("id_aurehal_avec_ville_extraite.xlsx", index=False)

# Réordonner les colonnes si besoin
cols_order = [
    'Centre', 'Equipe', 'Auteurs FR', 'Auteurs copubliants', 'Organisme copubliant',
    'Adresse', 'Ville', 'Pays', 'ID Aurehal', 'UE/Non UE', 'Année',
    'HalID', 'Domaine(s)', 'Mots-cles', 'Resume',"Latitude", "Longitude","geonameid"
]
if all(col in df_publis_tout.columns for col in cols_order):
    df_publis_tout = df_publis_tout[cols_order]


# Exporter le résultat principal
df_publis_tout.to_excel("resultat_avec_villes_completes_dico_et_crochets.xlsx", index=False)

# Temps de traitement, environ 12 secondes


In [ ]:
# On renseigne la latitude et la longitude des villes déjà connues dans le dictionnaire de référence, parmi celles qui ont été trouvées entre crochets

# On ne garde que le premier mot avant la virgule (ex: Richmond, UK --> Richmond)
df_publis_tout["Ville"] = df_publis_tout["Ville"].str.split(",").str[0].str.strip()


# Sélectionner les colonnes utiles dans df_villes
df_villes_geo = ""
df_publis_tout_geo = df_publis_tout

df_villes_geo = df_villes[["Ville", "Pays", "Latitude", "Longitude", "geonameid"]].drop_duplicates()

# Fusionner avec df_publis_tout sur Ville et Pays
df_temp = df_publis_tout_geo.merge(
    df_villes_geo,
    on=["Ville", "Pays"],
    how="left",
    suffixes=("_old", "_new")
)


# Mettre à jour Latitude/Longitude uniquement si elles sont vides
mask_lat = df_publis_tout_geo["Latitude"].isna() & df_temp["Latitude_new"].notna()
mask_lon = df_publis_tout_geo["Longitude"].isna() & df_temp["Longitude_new"].notna()

df_publis_tout_geo.loc[mask_lat, "Latitude"] = df_temp.loc[mask_lat, "Latitude_new"]
df_publis_tout_geo.loc[mask_lon, "Longitude"] = df_temp.loc[mask_lon, "Longitude_new"]

# mettre à jour geonameid si nécessaire
if "geonameid" in df_publis_tout_geo.columns:
    mask_geo = df_publis_tout_geo["geonameid"].isna() & df_temp["geonameid_new"].notna()
    df_publis_tout_geo.loc[mask_geo, "geonameid"] = df_temp.loc[mask_geo, "geonameid_new"]


# Exporter le résultat pour vérifier
df_publis_tout_geo.to_excel("Copublis_Inria_villes_a_completer.xlsx", index=False)

# Temps de traitement environ 15 secondes


In [ ]:
# inspection d'une référence pour contrôle visuel du résultat
df_publis_tout_geo[df_publis_tout_geo["HalID"] == "hal-03696264"]

,Centre,Equipe,Auteurs FR,Auteurs copubliants,Organisme copubliant,Adresse,Ville,Pays,ID Aurehal,UE/Non UE,Année,HalID,Domaine(s),Mots-cles,Resume,Latitude,Longitude,geonameid
27,Sophia,ABS,"Mazauric, Dorian","Chaintreau, Augustin",Columbia University [New York],"Columbia University in the City of New York, 2...",New York,United States,75524,Non UE,2022,hal-03696264,Computer Science [cs],NaN,We study a group-formation game on an undirect...,NaN,NaN,NaN


In [81]:
# Créer un fichier à part pour toutes les villes non trouvées

df_villes_vides = ""
# 1. Filtrer les lignes où Ville est vide
df_villes_vides = df_publis_tout_geo[df_publis_tout_geo["Ville"].isna()]

# 2. Grouper et agréger
df_villes_vides_uniques2 = df_villes_vides.groupby("ID Aurehal").agg({
    "Organisme copubliant": "first",
    "Adresse": "first",
    "Pays": "first",
    "HalID": lambda x: ", ".join(x.unique())
}).reset_index()

# 3. Exporter
df_villes_vides_uniques2.to_excel("villes_vides_uniques_aprs_dico_crochet_et_dico.xlsx", index=False)



In [ ]:
# Contrôle visuel du résultat
df.head(3)
